# 6- Un poquito de Spark

### 6.1 Comenzamos realizando la misma práctica que hicimos en Hive en Spark, importando el CSV. Sería recomentable intentarlo con opciones que quiten las "" de los campos, que ignoren los espacios innecesarios en los campos, que sustituyan los valores vacíos por 0 y que infiera el esquema. 

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
spark = (SparkSession
.builder
.enableHiveSupport()
.appName("Ejer6 ")
.getOrCreate())

In [3]:
spark.stop()

spark = (SparkSession
.builder
.enableHiveSupport()
.appName("Ejer6 ")
.getOrCreate())

In [4]:
df = (spark.read.format("csv")
    .option('header','true')
    .option('inferSchema','true')
    .option('delimiter',';')
    .load('rango.csv')
    .withColumn('DESC_DISTRITO',F.trim(F.col('DESC_DISTRITO')))
    .withColumn('DESC_BARRIO', F.trim(F.col('DESC_BARRIO')))
    .na.fill(0)
    )

df.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               7|               0|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|      

In [5]:
#df.select($"DESC_BARRIO").distinct().show()
df.select(F.col('DESC_BARRIO')).distinct().show()

+--------------------+
|         DESC_BARRIO|
+--------------------+
|        VALDEFUENTES|
|            ABRANTES|
|       LOS JERONIMOS|
|            VALVERDE|
|              CORTES|
|   PALOMERAS SURESTE|
|CIUDAD UNIVERSITARIA|
|      CUATRO VIENTOS|
|           TRAFALGAR|
|              HELLIN|
|    ALAMEDA DE OSUNA|
|          PRADOLONGO|
|            MOSCARDO|
|          VALDEZARZA|
|           RECOLETOS|
|             HORCAJO|
|        EL CAÑAVERAL|
|        VISTA ALEGRE|
|             EL VISO|
|    PUERTA DEL ANGEL|
+--------------------+
only showing top 20 rows



### 6.4 Crea una vista temporal de nombre "padron" y a través de ella cuenta el número de barrios diferentes que hay

In [6]:
df.createOrReplaceTempView("padron")

spark.sql("select count(distinct(cod_barrio)) from padron").show()

+--------------------------+
|count(DISTINCT cod_barrio)|
+--------------------------+
|                         9|
+--------------------------+



### 6.5 Crea una nueva columna que muestre la longitud de los campos de la columna  DESC_DISTRITO y que se llame "longitud".

In [7]:
df.withColumn("longitud",F.length(("DESC_DISTRITO"))).show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|longitud|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|       6|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               7|               0|                 1|                 0|       6|
|           1|       CENTRO|            101| 

### 6.6 Crea una nueva columna que muestre el valor 5 para cada uno de los registros de la tabla. 

In [8]:
dfcinco = df.withColumn("uncincoxd", F.lit(5)).show(5)

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+---------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|uncincoxd|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+---------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|        5|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               7|               0|                 1|                 0|        5|
|           1|       CENTRO|            

### 6.7 Borra esa columna

In [9]:
dfcinco = df.drop(F.col("uncincoxd"))

### 6.8 Particiona el DataFrame por las variables DESC_DISTRITO y DESC_BARRIO

In [10]:
dfpartition = df.repartition("DESC_DISTRITO","DESC_BARRIO")
print(dfpartition.rdd.getNumPartitions())

200


### 6.9 Almacénalo en caché. Consulta en el puerto 4040 (UI de Spark) de tu usuario local el estado de los rdds almacenados.

In [11]:
dfpartition.cache()

DataFrame[COD_DISTRITO: int, DESC_DISTRITO: string, COD_DIST_BARRIO: int, DESC_BARRIO: string, COD_BARRIO: int, COD_DIST_SECCION: int, COD_SECCION: int, COD_EDAD_INT: int, EspanolesHombres: int, EspanolesMujeres: int, ExtranjerosHombres: int, ExtranjerosMujeres: int]

### 6.10 Lanza una consulta contra el DF resultante en la que muestre el número total de "espanoleshombres", "espanolesmujeres", extranjeroshombres" y "extranjerosmujeres" para cada barrio de cada distrito. Las columnas distrito y barrio deben ser las primeras en aparecer en el show. Los resultados deben estar ordenados en orden de más a menos según la columna "extranjerosmujeres" y desempatarán por la columna "extranjeroshombres".

In [12]:
dfpartition.groupBy(F.col("DESC_DISTRITO"),F.col("DESC_BARRIO"))\
    .agg(F.sum(F.col("espanoleshombres")),F.sum(F.col("espanolesmujeres")),F.sum(F.col("extranjeroshombres")),F.sum("extranjerosmujeres"))\
        .orderBy(F.col("sum(extranjerosmujeres)").desc(),F.col("sum(extranjeroshombres)").desc())\
            .show(5)

+------------------+------------+---------------------+---------------------+-----------------------+-----------------------+
|     DESC_DISTRITO| DESC_BARRIO|sum(espanoleshombres)|sum(espanolesmujeres)|sum(extranjeroshombres)|sum(extranjerosmujeres)|
+------------------+------------+---------------------+---------------------+-----------------------+-----------------------+
|PUENTE DE VALLECAS|   SAN DIEGO|                13885|                15555|                   6871|                   7049|
|            LATINA|      ALUCHE|                24928|                29666|                   5425|                   6313|
|     CIUDAD LINEAL|PUEBLO NUEVO|                23255|                27261|                   5558|                   6291|
|       CARABANCHEL|VISTA ALEGRE|                15926|                19377|                   5496|                   6143|
|            CENTRO| EMBAJADORES|                16640|                16593|                   8214|                 

### 6.11 Elimina el registro en caché

In [13]:
spark.catalog.clearCache()

### 6.12 Crea un nuevo DataFrame a partir del original que muestre únicamente una columna con DESC_BARRIO, otra con DESC_DISTRITO y otra con el número total de "espanoleshombres" residentes en cada distrito de cada barrio. Únelo (con un join) con el DataFrame original a través de las columnas en común.

In [14]:
dfreduc = df.groupBy(F.col("DESC_BARRIO"),F.col("DESC_DISTRITO"))\
    .agg(F.sum(F.col("espanoleshombres")))

dfreduc.show(5)


+--------------+-------------------+---------------------+
|   DESC_BARRIO|      DESC_DISTRITO|sum(espanoleshombres)|
+--------------+-------------------+---------------------+
|       ACACIAS|         ARGANZUELA|                15399|
|      VALVERDE|FUENCARRAL-EL PARDO|                26922|
|FUENTELARREINA|FUENCARRAL-EL PARDO|                 1445|
|       PAVONES|          MORATALAZ|                 3584|
|     EL GOLOSO|FUENCARRAL-EL PARDO|                 8947|
+--------------+-------------------+---------------------+
only showing top 5 rows



In [15]:
rangoDF_join = df.alias("fst").join(dfreduc.alias("snd"),\
(F.col("fst.DESC_BARRIO") == F.col("snd.DESC_BARRIO")) & \
(F.col("fst.DESC_DISTRITO") == F.col("snd.DESC_DISTRITO")))

### 6.13 Repite la función anterior utilizando funciones de ventana. (over(Window.partitionBy.....))

In [17]:
dfreducnew = df.select(F.col("DESC_BARRIO"),F.col("DESC_DISTRITO"),\
    F.sum(F.col("espanoleshombres")).over(Window.partitionBy("DESC_BARRIO","DESC_DISTRITO")).alias("espanoleshombres"))

dfreducnew.show(5)

+-----------+-------------+----------------+
|DESC_BARRIO|DESC_DISTRITO|espanoleshombres|
+-----------+-------------+----------------+
|    ACACIAS|   ARGANZUELA|           15399|
|    ACACIAS|   ARGANZUELA|           15399|
|    ACACIAS|   ARGANZUELA|           15399|
|    ACACIAS|   ARGANZUELA|           15399|
|    ACACIAS|   ARGANZUELA|           15399|
+-----------+-------------+----------------+
only showing top 5 rows



In [18]:
rangoDF_join = df.alias("fst").join(dfreducnew.alias("snd"),\
(F.col("fst.DESC_BARRIO") == F.col("snd.DESC_BARRIO")) & \
(F.col("fst.DESC_DISTRITO") == F.col("snd.DESC_DISTRITO")))

rangoDF_join.show(5)

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+-----------+-------------+----------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|DESC_BARRIO|DESC_DISTRITO|espanoleshombres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+-----------+-------------+----------------+
|           2|   ARGANZUELA|            202|    ACACIAS|         2|            2085|         85|          26|               2|               1|                 0|                 1|    ACACIAS|   ARGANZUELA|           15399|
|           2|   ARGANZUELA|            202|    ACACIAS|         2|            2085|         85|    

### 6.14 Mediante una función Pivot muestra una tabla (que va a ser una tabla de contingencia) quecontenga los valores totales ()la suma de valores) de espanolesmujeres para cada distrito y en cada rango de edad (COD_EDAD_INT). Los distritos incluidos deben ser únicamente CENTRO, BARAJAS y RETIRO y deben figurar como columnas . El aspecto debe ser similar a este:

In [19]:
contingencia = df.groupBy("COD_EDAD_INT").pivot("DESC_DISTRITO").agg(F.sum(F.col("espanolesmujeres")))

contingencianew = contingencia.select(F.col("COD_EDAD_INT"),F.col("BARAJAS"),F.col("CENTRO"),F.col("RETIRO"))

### 6.15 Utilizando este nuevo DF, crea 3 columnas nuevas que hagan referencia a qué porcentaje de la suma de "espanolesmujeres" en los tres distritos para cada rango de edad representa cada uno de los tres distritos. Debe estar redondeada a 2 decimales. Puedes imponerte la condición extra de no apoyarte en ninguna columna auxiliar creada para el caso.

In [62]:
contingencianew.show()

+------------+-------+------+------+
|COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|
+------------+-------+------+------+
|          31|    191|   860|   611|
|          85|    105|   349|   531|
|          65|    215|   631|   904|
|          53|    354|   781|   774|
|          78|    202|   457|   699|
|         108|   null|     1|  null|
|          34|    241|   813|   618|
|         101|      4|    16|    20|
|          81|    189|   440|   678|
|          28|    155|   811|   656|
|          76|    250|   461|   782|
|          27|    146|   738|   543|
|          26|    181|   668|   535|
|          44|    380|   849|   788|
|         103|      1|     6|     9|
|          12|    263|   255|   433|
|          91|     58|   234|   329|
|          22|    202|   324|   427|
|          93|     30|   191|   210|
|         111|   null|     2|  null|
+------------+-------+------+------+
only showing top 20 rows



In [67]:
# Obtener total mujeres



totales = contingencianew.select(F.sum(F.col("BARAJAS")),F.sum(F.col("CENTRO")),F.sum(F.col("RETIRO"))).collect()

# otra posibilidad: withcolumn que sume las otras 3 (mejor)
suma = totales[0][0] + totales[0][1] + totales [0][2]

print(suma)

133736


In [68]:
print(suma)

contingencianew.withColumn("PORCENTAJEBARAJAS",F.round((F.col("BARAJAS")/suma*100),2)).show()

133736
+------------+-------+------+------+-----------------+
|COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|PORCENTAJEBARAJAS|
+------------+-------+------+------+-----------------+
|          31|    191|   860|   611|             0.14|
|          85|    105|   349|   531|             0.08|
|          65|    215|   631|   904|             0.16|
|          53|    354|   781|   774|             0.26|
|          78|    202|   457|   699|             0.15|
|         108|   null|     1|  null|             null|
|          34|    241|   813|   618|             0.18|
|         101|      4|    16|    20|              0.0|
|          81|    189|   440|   678|             0.14|
|          28|    155|   811|   656|             0.12|
|          76|    250|   461|   782|             0.19|
|          27|    146|   738|   543|             0.11|
|          26|    181|   668|   535|             0.14|
|          44|    380|   849|   788|             0.28|
|         103|      1|     6|     9|              0.0|
|  